In [2]:
from functools import reduce
from py_ecc.bn128 import G1, G2, bn128_curve, curve_order, multiply, neg, pairing, final_exponentiate
from sage.all import *

F = GF(curve_order)
F_x.<x> = F[] # polynomial over finite field

A = Matrix(F, [
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0]
])

B = Matrix(F, [
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1]
])

C = Matrix(F,[
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [0,1,0,0,0,0,0,0]
])

x = F.random_element()
y = F.random_element()
z = F.random_element()
u = F.random_element()
l = 2 # public variables index


v1 = x * y
v2 = z * u
out = v1 * v2

# create the witness vector
witness = vector(F, [1, out, x, y, z, u, v1, v2])

assert Matrix(C * witness) == Matrix(A * witness).elementwise_product(Matrix(B * witness))

def bn_add(A, B):
    return bn128_curve.add(A, B)

def trusted_setup(n, m, t_x, U_polys, V_polys, W_polys):
    encrypted_G1 = list()
    encrypted_G2 = list()
    t_x_enc = list()

    tau = F.random_element()
    pow_of_tau = 1  # tau ** 0

    alpha = F.random_element()
    beta = F.random_element()
    gamma = F.random_element()
    delta = F.random_element()
    gamma_inv = inverse_mod(gamma, curve_order)
    delta_inv = inverse_mod(delta, curve_order)

    Alpha1 = multiply(G1, int(alpha))
    Beta1 = multiply(G1, int(beta))
    Beta2 = multiply(G2, int(beta))
    
    Gamma2 = multiply(G2, int(gamma))
    Delta1 = multiply(G1, int(delta))
    Delta2 = multiply(G2, int(delta))
    t_x_eval = t_x(tau)

    pow_of_tau_C_private = list()
    pow_of_tau_C_public = list()

    for i in range(0, n):
        encrypted_G1.append(multiply(G1, int(pow_of_tau)))
        encrypted_G2.append(multiply(G2, int(pow_of_tau)))

        if i <= n-2:
            t_x_enc.append(multiply(G1, int(pow_of_tau * t_x_eval * delta_inv)))

        pow_of_tau = pow_of_tau*tau

    U_polys_enc = encrypt_polynomials(U_polys, encrypted_G1)
    V_polys_enc = encrypt_polynomials(V_polys, encrypted_G1)
    W_polys_enc = encrypt_polynomials(W_polys, encrypted_G1)


    for i in range(0, l):
        pow_of_tau_C_public.append(multiply(bn_add(bn_add(multiply(U_polys_enc[i], int(beta)), multiply(V_polys_enc[i], int(alpha))), W_polys_enc[i]), gamma_inv))

    for i in range(l, m):
        pow_of_tau_C_private.append(multiply(bn_add(bn_add(multiply(U_polys_enc[i], int(beta)), multiply(V_polys_enc[i], int(alpha))), W_polys_enc[i]), delta_inv))


    return encrypted_G1, encrypted_G2, t_x_enc, Alpha1, Beta1, Beta2, Gamma2, Delta1, Delta2, pow_of_tau_C_public, pow_of_tau_C_private

def generate_target_polynomial(n):
    # Create a polynomial of the form (x - 1)(x - 2)(x - 3)...(x - n)
    x = var('x')
    res = F_x(prod([(x - i) for i in range(1, n+1)]))
    for i in range(1, n+1):
        assert res(i) == 0, "Invalid target polynom"
    return res

def interpolate_polynomials(matrix):
    result = list()
    for column in matrix.columns():
        polynomial = F_x.lagrange_polynomial([(index+1, values) for index, values in enumerate(column)])
        result.append(polynomial)
    return result

def inner_product_polynomials_with_witness(polys, witness):
    def mul(x, y): return x * y
    def sum(x, y): return x + y
    return reduce(sum, map(mul, polys, witness))

def inner_product_enc_polynomials_with_witness(polys, witness):
    def mul(x, y): return multiply(x, int(y))
    return reduce(bn_add, map(mul, polys, witness))

def encrypt_polynomials(polynomials, encrypted_points):
    encrypted_polynomials = [None] * len(polynomials)
    for i, poly in enumerate(polynomials):
        if poly != 0:
            for j, coeff in enumerate(poly.coefficients(sparse=False)):
                encrypted_polynomials[i] = bn_add(encrypted_polynomials[i], multiply(encrypted_points[j], int(coeff)))
    return encrypted_polynomials

def encrypt_hx_tx(h_x, t_x_enc):
    h_x_enc = list()
    for index, coeff in enumerate(h_x.coefficients(sparse=False)):
        h_x_enc.append(multiply(t_x_enc[index], int(coeff)))
    return reduce(bn_add, h_x_enc)

t_x = generate_target_polynomial(len(A.rows()))
U_polys = interpolate_polynomials(A)
V_polys = interpolate_polynomials(B)
W_polys = interpolate_polynomials(C)

(p_tau_G1, p_tau_G2, t_x_enc, Alpha1, Beta1, Beta2, Gamma2, Delta1, Delta2, pow_of_tau_C_public, pow_of_tau_C_private) = \
    trusted_setup(len(A.rows()), len(A.columns()), t_x, U_polys, V_polys, W_polys)

poly1 = inner_product_polynomials_with_witness(U_polys, witness)
poly2 = inner_product_polynomials_with_witness(V_polys, witness)
poly3 = inner_product_polynomials_with_witness(W_polys, witness)  

h_x, remainder = ((poly1 * poly2) - poly3).quo_rem(t_x)
assert remainder == 0, "Division has a remainder"

hx_tx_enc1 = encrypt_hx_tx(h_x, t_x_enc)

A1_inner_prod = inner_product_enc_polynomials_with_witness(encrypt_polynomials(U_polys, p_tau_G1), witness)
B1_inner_prod = inner_product_enc_polynomials_with_witness(encrypt_polynomials(V_polys, p_tau_G1), witness)
B2_inner_prod = inner_product_enc_polynomials_with_witness(encrypt_polynomials(V_polys, p_tau_G2), witness)
C1_inner_prod_public = inner_product_enc_polynomials_with_witness(pow_of_tau_C_public, witness[:l])
C1_inner_prod_private = inner_product_enc_polynomials_with_witness(pow_of_tau_C_private, witness[l:])

r = F.random_element()
s = F.random_element()

rDelta1 = multiply(Delta1, int(r))
sDelta1 = multiply(Delta1, int(s))
sDelta2 = multiply(Delta2, int(s))

A1 = bn_add(bn_add(A1_inner_prod, Alpha1), rDelta1)
B1 = bn_add(bn_add(B1_inner_prod, Beta1), sDelta1)
B2 = bn_add(bn_add(B2_inner_prod, Beta2), sDelta2)

sA1 = multiply(A1, int(s))
rB1 = multiply(B1, int(r))
rsDelta1 = multiply(Delta1, int(r*s))
C1 = bn_add(bn_add(hx_tx_enc1, C1_inner_prod_private), bn_add(sA1, bn_add(rB1, neg(rsDelta1))))

assert final_exponentiate(pairing(B2, A1) ) == final_exponentiate(
                    pairing(Beta2, Alpha1) * \
                    pairing(Gamma2, C1_inner_prod_public) * \
                    pairing(Delta2, C1)\
                ), "Equality fail"

print("Success")


Success
